In [138]:
from bs4 import BeautifulSoup
from datetime import datetime
import pandas as pd
import requests, re

In [162]:
jobs = []
for pag in range(1, 43):
    res = requests.get('https://stackoverflow.com/jobs?med=site-ui&ref=jobs-tab&sort=i&pg=%d'%pag)
    soup = BeautifulSoup(res.text, 'lxml')
    
    # Jobs list html
    jobs_html = soup.find('div', class_='listResults').findAll('div', class_='-job')
    for job_html in jobs_html:
        # job title
        title_html = job_html.find('a', class_='job-link')
        job = {'title': title_html.text if title_html else None}
        
        # job company and location
        company, location = tuple(job_html.find('div', class_='-company').select('span'))
        job['company'], job['location'] = company.text, location.text
        
        # job posted
        job['posted'] = job_html.find('div', class_='-title').find('span', class_='pt2').text
        
        # request date
        job['request_date'] = datetime.now()
        
        # job salary and perks
        perks_html = job_html.find('div', class_='-perks')
        if perks_html:
            perks = []
            for p in perks_html.select('span'):

                if '-salary' in str(p):
                    job['salary'] = p.text.strip()
                else:
                    perks.append(p.text.strip())

            job['perks'] = ', '.join(perks)
            
        
        # job tags
        tags_html = job_html.find('div', class_='-tags')
        if tags_html:
            tags = [a.text for a in tags_html.select('a')]
            job['tags'] = ', '.join(tags)  
        
        jobs.append(job)
    

In [163]:
df = pd.DataFrame(jobs)

In [164]:
df.shape

(1050, 8)

In [165]:
df.head(10)

,company,location,perks,posted,request_date,salary,tags,title
0,Corelight,"\r\n - \r\nSan Francisco, CA",Visa sponsor,5h ago,2018-11-13 02:38:16.258048,$120k - 170k\r\n\r\n |...,"user-interface, reactjs, go, node.js, rest",user-interface
1,Lux Group,"\r\n - \r\nSydney, Australia",,< 1h ago,2018-11-13 02:38:16.258048,A$120k - 150k,"web-services, javascript, reactjs, node.js",web-services
2,TFG Co.,"\r\n - \r\nState of São Paulo, Bra...",NaN,< 1h ago,2018-11-13 02:38:16.258048,NaN,"spark, presto, hive, kubernetes, kafka",spark
3,Wallethub,\r\n - \r\nNo office location,Remote,2h ago,2018-11-13 02:38:16.258048,NaN,"spring, java, spring-mvc, java-ee",spring
4,Wallethub,\r\n - \r\nNo office location,Remote,2h ago,2018-11-13 02:38:16.258048,NaN,"selenium, java, automation, automated-tests, t...",selenium
5,Presence,"\r\n - \r\nSaint Petersburg, FL ...",Paid relocation,3h ago,2018-11-13 02:38:16.258048,$90k - 120k\r\n\r\n | ...,"angularjs, .net, c#, .net-core, javascript",angularjs
6,ClearPoint Federal Bank & Trust,"\r\n - \r\nBatesville, IN",Remote,3h ago,2018-11-13 02:38:16.273671,$90k - 110k,"c#, ssms-2017, javascript, reactjs, git",c#
7,Manulife / John Hancock,"\r\n - \r\nKitchener, ON, Canada ...",On-site and limited remote,3h ago,2018-11-13 02:38:16.273671,C$85k - 115k,"java, javascript, c#, html5, react-relay",java
8,S2P Project Professionals,"\r\n - \r\nSydney, Australia",,4h ago,2018-11-13 02:38:16.273671,A$130k - 180k,"amazon-web-services, java, automation, go, ans...",amazon-web-services
9,Corelight,"\r\n - \r\nSan Francisco, CA",Visa sponsor,4h ago,2018-11-13 02:38:16.273671,Equity,"qa, python, testing, bash, continuous-integration",qa


In [166]:
df.to_csv('Stack_Overflow_Jobs.csv', index=False)